In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4043
SparkContext available as 'sc' (version = 2.3.2, master = local[*], app id = local-1559038530594)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7e6bcbc


In [2]:
%%python
from pyspark.ml.linalg import Vectors
import numpy as np

In [3]:
%%python
# Load MNIST
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
#print(x_train.shape, x_test.shape)
x_train=x_train.flatten().reshape(60000, 28*28)
x_test=x_test.flatten().reshape(10000, 28*28)
y_train=y_train.flatten().reshape(60000, 1)
y_test=y_test.flatten().reshape(10000, 1)

In [4]:
%%python
x_train.shape

(60000, 784)

In [6]:
%%python
y_x_tr = np.hstack([y_train, x_train])
dff = map(lambda y_x: (
                    int(y_x[0]), Vectors.dense(y_x[1:])
                    ),  y_x_tr    
         )

mnistdf = spark.createDataFrame(dff, schema=["label", "features"]).cache()
#mnistdf = spark.createDataFrame(, schema=["label", "features"])

In [ ]:
%%python
mnistdf.take(1)

2019-05-28 12:16:23 WARN  TaskSetManager:66 - Stage 1 contains a task of very large size (49583 KB). The maximum recommended task size is 100 KB.


In [60]:
%%python
print(x_train.shape)
print(y_train.shape)
x_y_tr = np.concatenate(x_train, y_train)

(60000, 784)
(60000, 1)


TypeError: ('only integer scalar arrays can be converted to a scalar index',)

In [25]:
%%python
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train, y_train)

In [40]:
%%python
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
enc.fit(y_test)
pred = clf.predict_proba(x_test)
pred_cond = enc.inverse_transform(pred)

#y_te_exp = enc.transform(y_test)

print(pred[:1])
print(metrics.accuracy_score(y_test, pred_cond))

[[3.72652085e-05 2.05144996e-12 9.79531658e-05 1.64911078e-02
  1.65876375e-06 3.25361582e-04 3.54333509e-09 9.82327820e-01
  5.20158624e-05 6.66813810e-04]]
0.9201


In [ ]:
%%python
from sklearn.